In [1]:
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Loading data
file_path = Path("Resources/prepped_cardio_data.csv")
cleaned_df = pd.read_csv(file_path)
cleaned_df = cleaned_df.drop(columns=["index"])
cleaned_df.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Diabetes_num,Arthritis_Binary,Alcohol_outlier_treated,Fruit_outlier_treated,Green_Vegetables_outlier_treated,FriedPotato_outlier_treated
0,0,3,0,0,0,0,0,0,0,150.0,32.66,14.54,1,0,1,0.0,30.0,16.0,12.0
1,3,4,0,1,0,0,0,0,0,165.0,77.11,28.29,0,1,0,0.0,30.0,0.0,4.0
2,3,4,1,0,0,0,0,0,1,163.0,88.45,33.47,0,1,0,4.0,12.0,3.0,16.0
3,0,4,1,1,0,0,0,1,2,180.0,93.44,28.73,0,1,0,0.0,30.0,30.0,8.0
4,2,4,0,0,0,0,0,1,3,191.0,88.45,24.37,1,0,0,0.0,8.0,4.0,0.0


In [3]:
test_df = cleaned_df.copy()
test1_df = test_df[test_df['Heart_Disease']==1]
test_df = test_df[test_df['Heart_Disease']==0]
test_df = test_df.sample(n=24971)
test_df = pd.concat([test_df, test1_df]).sample(frac=1)
test_df

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Diabetes_num,Arthritis_Binary,Alcohol_outlier_treated,Fruit_outlier_treated,Green_Vegetables_outlier_treated,FriedPotato_outlier_treated
162830,3,4,1,0,0,0,0,0,7,157.0,54.430,21.95,0,0,0,1.0,12.0,6.0,5.0
172820,2,4,1,1,0,0,0,1,11,185.0,104.330,30.34,1,0,0,15.0,28.0,8.0,4.0
118856,3,4,1,1,0,0,0,0,2,168.0,68.040,24.21,1,1,1,0.0,30.0,30.0,4.0
113172,2,4,0,0,0,0,0,1,0,170.0,135.620,43.31,1,0,1,15.0,4.0,0.0,1.0
163132,4,4,0,1,0,0,0,0,4,163.0,68.950,26.09,1,0,1,0.0,28.0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184832,4,4,1,1,0,1,1,0,9,163.0,81.650,30.90,1,1,1,0.0,57.0,30.0,0.0
55035,0,4,0,0,0,1,1,0,5,178.0,69.400,21.95,1,0,1,0.0,30.0,8.0,2.0
278725,3,4,1,1,0,0,0,0,9,170.0,97.520,33.67,0,1,1,0.0,57.0,30.0,5.0
225597,4,3,1,0,0,0,0,0,10,170.0,63.500,21.93,0,0,0,8.0,57.0,8.0,8.0


In [4]:
# Define features set
X = test_df.copy()
X.drop("Heart_Disease", axis=1, inplace=True)
X.head()

,General_Health,Checkup,Exercise,Skin_Cancer,Other_Cancer,Depression,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Diabetes_num,Arthritis_Binary,Alcohol_outlier_treated,Fruit_outlier_treated,Green_Vegetables_outlier_treated,FriedPotato_outlier_treated
162830,3,4,1,0,0,0,0,7,157.0,54.43,21.95,0,0,0,1.0,12.0,6.0,5.0
172820,2,4,1,0,0,0,1,11,185.0,104.33,30.34,1,0,0,15.0,28.0,8.0,4.0
118856,3,4,1,0,0,0,0,2,168.0,68.04,24.21,1,1,1,0.0,30.0,30.0,4.0
113172,2,4,0,0,0,0,1,0,170.0,135.62,43.31,1,0,1,15.0,4.0,0.0,1.0
163132,4,4,0,0,0,0,0,4,163.0,68.95,26.09,1,0,1,0.0,28.0,4.0,8.0


In [5]:
# Define target vector
y = test_df["Heart_Disease"].values.reshape(-1, 1)
y[:5]

array([[0],
       [1],
       [1],
       [0],
       [1]], dtype=int64)

In [6]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [7]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [8]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [9]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
import numpy as np
print(X_train.isnull().sum())
nan_in_y = np.isnan(y_train).any()
print(f'NaN values in y_train: {nan_in_y}')

General_Health                      0
Checkup                             0
Exercise                            0
Skin_Cancer                         0
Other_Cancer                        0
Depression                          0
Sex                                 0
Age_Category                        0
Height_(cm)                         0
Weight_(kg)                         0
BMI                                 0
Smoking_History                     0
Diabetes_num                        0
Arthritis_Binary                    0
Alcohol_outlier_treated             0
Fruit_outlier_treated               0
Green_Vegetables_outlier_treated    0
FriedPotato_outlier_treated         0
dtype: int64
NaN values in y_train: False


In [20]:
# from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': [None, 'sqrt', 'log2']
}

# grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, error_score='raise')
# grid_search.fit(X_train, y_train)
# best_model = grid_search.best_estimator_

random_search = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=param_grid, n_iter=100, cv=5, random_state=42)
random_search.fit(X_train, y_train)
best_model_random = random_search.best_estimator_

## Fitting the Decision Tree Model

In [21]:
# Creating the decision tree classifier instance
# model = tree.DecisionTreeClassifier()

In [22]:
# Fitting the model
# model = model.fit(X_train_scaled, y_train)

## Making Predictions Using the Tree Model

In [23]:
test_score = best_model_random.score(X_test, y_test)
print(f'Test Score: {test_score}')

Test Score: 0.7443536761172513


In [24]:
# Making predictions using the testing data
# predictions = model.predict(X_test_scaled)

In [25]:
predictions = best_model_random.predict(X_test)

## Model Evaluation

In [26]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [27]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4205,2031
Actual 1,1161,5089


Accuracy Score : 0.7443536761172513
Classification Report
              precision    recall  f1-score   support

           0       0.78      0.67      0.72      6236
           1       0.71      0.81      0.76      6250

    accuracy                           0.74     12486
   macro avg       0.75      0.74      0.74     12486
weighted avg       0.75      0.74      0.74     12486



test_df Random - Confusion Matrix
Predicted 0	Predicted 1
Actual 0	4314	1920
Actual 1	1262	4990
Accuracy Score : 0.7451545731218965
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.69      0.73      6234
           1       0.72      0.80      0.76      6252

    accuracy                           0.75     12486
   macro avg       0.75      0.75      0.74     12486
weighted avg       0.75      0.75      0.74     12486

test_df grid searh - Confusion Matrix
Predicted 0	Predicted 1
Actual 0	4314	1920
Actual 1	1262	4990
Accuracy Score : 0.7451545731218965
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.69      0.73      6234
           1       0.72      0.80      0.76      6252

    accuracy                           0.75     12486
   macro avg       0.75      0.75      0.74     12486
weighted avg       0.75      0.75      0.74     12486

scaled random test_df - Confusion Matrix
Predicted 0	Predicted 1
Actual 0	4205	2031
Actual 1	1161	5089
Accuracy Score : 0.7443536761172513
Classification Report
              precision    recall  f1-score   support

           0       0.78      0.67      0.72      6236
           1       0.71      0.81      0.76      6250

    accuracy                           0.74     12486
   macro avg       0.75      0.74      0.74     12486
weighted avg       0.75      0.74      0.74     12486